In [9]:
import mcdm
import numpy as np
from pymcdm.methods import TOPSIS
from pymcdm.methods import VIKOR
from pymcdm.methods import PROMETHEE_II
from pymcdm.helpers import rankdata, rrankdata
from pymcdm import weights as mcdm_weights

In [10]:
from prettytable import from_csv
with open("pima_perform.tsv") as fp:
    mytable = from_csv(fp)
print(mytable)

+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
|                      | ACCURACY | PRECISION | RECALL |     AUC     | F1SCORE |   FP_RATE   |   FN_RATE   | SPECIFICITY | HAMMING LOSS | ENTROPY LOSS |
+----------------------+----------+-----------+--------+-------------+---------+-------------+-------------+-------------+--------------+--------------+
| LOGISTIC REGRESSION  |   0.82   |    0.82   |  0.82  | 0.864458405 |   0.81  | 0.067961165 | 0.411764706 | 0.932038835 | 0.181818182  | 6.279813872  |
|    DECISION TREE     |   0.7    |    0.71   |  0.7   | 0.672758424 |   0.7   | 0.242718447 | 0.411764706 | 0.757281553 | 0.298701299  | 10.31690717  |
| GAUSSIAN NAIVE BAYES |   0.82   |    0.82   |  0.82  | 0.885018085 |   0.82  | 0.097087379 | 0.333333333 | 0.902912621 | 0.902912621  | 6.055551679  |
|         SVM          |   0.48   |    0.46   |  0.48  | 0.726537217 |   0.47  | 0

In [11]:
x_matrix, alt_names = mcdm.load("pima_perform.tsv", delimiter="\t", skiprows=1, labeled_rows=True)
alts = np.array(x_matrix, dtype='float')
weights = mcdm_weights.equal(alts)
weight=weights.tolist()
types = np.array([1,1,1,1,1,-1,-1,1,-1,-1])

In [12]:
topsis = TOPSIS()
# Determine preferences and ranking for alternatives
pref1 = topsis(alts, weights, types)
ranking1= rrankdata(pref1)

for r, p in zip(ranking1, pref1):
    print(r, p)


1.0 0.8629338746592019
4.0 0.7068986582655924
3.0 0.7284986681275627
6.0 0.410941544822066
5.0 0.6616393124165876
2.0 0.8197514116393139
7.0 0.2655433900068337


In [13]:
vikor= VIKOR()
pref2 = vikor(alts, weights, types)
ranking2= rankdata(pref2)

for r, p in zip(ranking2, pref2):
    print(r, p)

1.0 0.0
3.0 0.2017899391358467
4.0 0.5545978623708471
6.0 0.8404849051364351
5.0 0.5549592340026035
2.0 0.09646822887856887
7.0 1.0


In [14]:
promethe=PROMETHEE_II('usual')
p = np.random.rand(x_matrix.shape[1]) / 2
q = np.random.rand(x_matrix.shape[1]) / 2 + 0.5
pref3 = promethe(alts, weights, types,p=p,q=q)
ranking3 = rrankdata(pref3)

for r, p in zip(ranking3, pref3):
    print(r, p)

1.0 0.7333333333333332
5.0 -0.16666666666666663
2.0 0.5833333333333333
6.0 -0.5999999999999999
4.0 -0.01666666666666672
3.0 0.23333333333333328
7.0 -0.7666666666666665


In [15]:
from prettytable import PrettyTable
print("#########################PIMA DATASET#######################")
columns = ["Model", "TOPSIS","Value","VIKOR","VALUES","PROMETHEE","VALUES"]
Table = PrettyTable()
Table.add_column(columns[0], ["LOGISTIC REGRESSION ", "DECISION TREE", "GAUSSIAN NAIVE BAYES",
                       "SVM ", "NEURAL NETWORK", "RANDOM FOREST ", "LINEAR MODEL"])
Table.add_column(columns[1], ranking1)
Table.add_column(columns[2], pref1)
Table.add_column(columns[3], ranking2)
Table.add_column(columns[4], pref2)
Table.add_column(columns[5], ranking3)
Table.add_column(columns[6], pref3)

print(Table)

#########################PIMA DATASET#######################
+----------------------+--------+--------------------+-------+---------------------+-----------+----------------------+
|        Model         | TOPSIS |       Value        | VIKOR |        VALUES       | PROMETHEE |        VALUES        |
+----------------------+--------+--------------------+-------+---------------------+-----------+----------------------+
| LOGISTIC REGRESSION  |  1.0   | 0.8629338746592019 |  1.0  |         0.0         |    1.0    |  0.7333333333333332  |
|    DECISION TREE     |  4.0   | 0.7068986582655924 |  3.0  |  0.2017899391358467 |    5.0    | -0.16666666666666663 |
| GAUSSIAN NAIVE BAYES |  3.0   | 0.7284986681275627 |  4.0  |  0.5545978623708471 |    2.0    |  0.5833333333333333  |
|         SVM          |  6.0   | 0.410941544822066  |  6.0  |  0.8404849051364351 |    6.0    | -0.5999999999999999  |
|    NEURAL NETWORK    |  5.0   | 0.6616393124165876 |  5.0  |  0.5549592340026035 |    4.0    | -0

In [16]:
ans=mcdm.rank(x_matrix, alt_names=alt_names, n_method="Linear2", is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight, s_method="SAW") 
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################SAW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table) 

#########################SAW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
| GAUSSIAN NAIVE BAYES |  0.7380952380605442 |
| LOGISTIC REGRESSION  |  0.6421000585585023 |
|    RANDOM FOREST     |  0.6103553813173613 |
|    NEURAL NETWORK    |  0.5707770190750434 |
|    DECISION TREE     |  0.5377209005145418 |
|         SVM          |  0.487722733859536  |
|     LINEAR MODEL     | 0.26753803117319563 |
+----------------------+---------------------+


In [17]:
ans=mcdm.rank(x_matrix,alt_names=alt_names,n_method="Linear1",is_benefit_x=[1,1,1,1,1,-1,-1,1,-1,-1], w_vector=weight,s_method="MEW")
model=[]
value=[]
for x in ans:
    model.append(x[0])
    value.append(x[1])

print("#########################MEW#######################")
columns = ["Model","Value"]
Table = PrettyTable()
Table.add_column(columns[0], model)
Table.add_column(columns[1], value)

print(Table)

#########################MEW#######################
+----------------------+---------------------+
|        Model         |        Value        |
+----------------------+---------------------+
|         SVM          |  0.6314564050822976 |
| GAUSSIAN NAIVE BAYES |  0.6299574722959361 |
|    DECISION TREE     |  0.5983600099042867 |
|    RANDOM FOREST     |  0.5703207851032933 |
|    NEURAL NETWORK    |  0.5674777474829648 |
| LOGISTIC REGRESSION  |  0.5306420695139601 |
|     LINEAR MODEL     | 0.26804802138735534 |
+----------------------+---------------------+
